In [1]:
import math
import numpy as np
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import random

In [2]:
import keras
from keras.models import Sequential
from keras.layers import Convolution2D 
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

In [3]:
### INIT CNN ###
vector = np.array((0,0,0))
###########
### CNN ###
###########
inputshape = vector.shape
model2 = Sequential()
model2.add(Dense(12,input_dim=3,activation='relu'))
model2.add(Dense(8,activation='relu'))
model2.add(Dense(6,activation='relu'))
model2.add(Dense(units=1,activation='sigmoid')) #min_value=0, max_value=1
#model2.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
#model2.compile(loss='CategoricalCrossentropy',optimizer='adam',metrics=['accuracy'])
model2.compile(loss='mean_squared_error',optimizer='adam',metrics=['mse'])
print("the prediction for vector is: ",model2.predict(vector.reshape(1,3)))

the prediction for vector is:  [[0.5]]


In [4]:
pole_length = 0.3 #[m]
angle=0
time_interval = 0.3 #[sec]
g=9.8 #[m/s^2]
ball_mass = 0.005 #[kg]
ball_radius = 0.025 #[m]
friction_coefficient = 0.2
v0 = 0
x0 = 0 #[m]
min_angle=-8 #[deg]
max_angle=-8 #[deg]
height,width=240,640
length = 610
image = np.ones((height, width)) * 255


sucess_time = 30 #[sec]
gamma = 0.9
wins = 0
loss = 0
x_train,y_train = [],[]
i_step_in_current_round = 0 
i_step_in_total_from_beginning = 0 
num_of_round = 0 



In [5]:
def predict2degree(degreeAsFloat):
    mydegree = int((degreeAsFloat-0.5)*16)
    return mydegree

In [6]:
def drawImage(ballPosition,lineAngle):
    #draw line
    image = np.ones((height, width)) * 255
    normalize_x = ballPosition
    angle = lineAngle
    x_center, y_center = int(width/2), int(height/2)
    x2 = x_center+int(length/2*math.cos(angle))
    y2 = y_center+int(length/2*math.sin(angle))
    x1,y1 = (x_center-int(length/2*math.cos(angle))),(y_center-int(length/2*math.sin(angle)))
    line_thickness = 2
    cv2.line(image, (x1, y1), (x2, y2), (0, 255, 0), thickness=line_thickness)
    
    M = (y2-y1)/(x2-x1)
    y_center_of_circle = M*normalize_x + y1 -M*x1
    
    #draw circle
    radius = 10
    center_coordinates = (int(normalize_x),int(y_center_of_circle+radius))
    color = (0, 255, 0)
    thickness = 2
    image = cv2.circle(image, center_coordinates, radius, color, thickness)
        
    image = np.flip(image)
    return image

In [7]:
degree = predict2degree(model2.predict(vector.reshape(1,3)))
x_train,y_train = [],[]

print("starting")
num_of_round=0
while(num_of_round<100):
    num_of_round+=1
    print("num_of_round",num_of_round)
    mystack = [np.ones((height, width)) * 255]
    for i in range(0,100):
        if(random.random() < math.exp(-num_of_round/10)):
            degree=random.randint(-8, 8+1)
            print("degree was randomize:",degree)
        angle=math.pi*(degree/180)
        acceleration = (3/5)*g*math.sin(angle)
        
        current_v = v0 + acceleration*time_interval 
        current_x = x0 + v0*time_interval + (1/2)*acceleration*(time_interval**2)
        
        
        v0 = current_v
        x0 = current_x
        #print("after",time_interval,"[sec], at angle=",angle*180/math.pi,"[deg], the ball moved to ",current_x)
    
        normalize_x = 300-current_x*300/1.5
        if((normalize_x > 600) or (normalize_x < 10)):
            break
    
        
        image = drawImage(normalize_x,angle)
        mystack = np.concatenate((mystack,[image]),axis=0)
        
        vector = np.array((degree/8,normalize_x/600,current_v/10))
        model_prediction = model2.predict(vector.reshape(1,3))
        degree = predict2degree(model_prediction)

        x_train.append(vector)
        y_train.append(model_prediction)
        print("vector",vector)
        print("degree",degree)
        
    print("i",i)          
    x0=random.random()/5
    v0=random.random()/10
    
    if(i*time_interval >= sucess_time):
        print("model ended without ball falling")
    elif((normalize_x > 600) or (normalize_x < 10)):
        print("game ended becouse ball dropped")
        print("len(y_train)",len(y_train))
        if(len(y_train)>=12):
            for j in range(1,5,1): 
                    y_train[len(y_train)-j] *= (-1)
        else:
            for j in range(1,int(len(y_train)*0.8),1): 
                    y_train[len(y_train)-j] *= (-1)
    model2.fit(x=np.vstack(x_train),y=np.vstack(y_train))
    x_train,y_train = [],[]
    

starting
num_of_round 1
degree was randomize: -6
vector [-0.75        0.50921941 -0.01843882]
degree 0
degree was randomize: -8
vector [-1.          0.5399333  -0.04298896]
degree 0
degree was randomize: 4
vector [ 0.5         0.57676973 -0.03068391]
degree 0
degree was randomize: -6
vector [-0.75        0.61667306 -0.04912273]
degree 0
degree was randomize: 8
vector [ 1.          0.65352073 -0.0245726 ]
degree 0
degree was randomize: 1
vector [ 0.125       0.67655402 -0.021494  ]
degree 0
vector [ 0.          0.69804802 -0.021494  ]
degree 0
degree was randomize: -5
vector [-0.625       0.72722915 -0.03686827]
degree 0
vector [ 0.          0.76409742 -0.03686827]
degree 0
degree was randomize: 6
vector [ 0.75        0.79174628 -0.01842945]
degree 0
degree was randomize: 7
vector [0.875      0.79942685 0.0030683 ]
degree 0
degree was randomize: -5
vector [-0.625       0.80404568 -0.01230597]
degree 0
degree was randomize: -5
vector [-0.625       0.82403878 -0.02768024]
degree 0
degree 

In [11]:
import imageio; 
from IPython.display import Video; 
edges = mystack
imageio.mimwrite('test2.mp4', edges, fps=7); 
Video('test2.mp4', width=640, height=480)

Lossy conversion from float64 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.


In [9]:
x_train,y_train = [],[]
vector = np.array((4/8,100/600,6/10))
#int((model2.predict(vector.reshape(1,3))-0.5)*8)
degree = predict2degree(model2.predict(vector.reshape(1,3)))
print(degree)




-1


In [10]:
x_train,y_train = [],[]
print("vector:",vector)
print("degree:",degree)
x_train.append(vector)
y_train.append(-1*(degree/16+0.5))
print(y_train)
model2.fit(x=np.vstack(x_train),y=np.vstack(y_train))

vector: [0.5        0.16666667 0.6       ]
degree: -1
[-0.4375]
1/1 [==============================] - 0s 3ms/step - loss: 0.6646 - mse: 0.6646
